In [ ]:
## set covid19_datasciente as path for python find bulletin package
import sys
from pathlib import Path
import os
from os import getcwd, remove
from os.path import join, basename

sys.path.append(str(Path(getcwd()).parent.parent))

In [ ]:
def macro_sort(column):
    """Sort function"""
    macros = ['Leste', 'Oeste', 'Noroeste', 'Norte', 'Fora','Total']
    correspondence = {macro: order for order, macro in enumerate(macros)}
    return column.map(correspondence)

In [ ]:
import pandas as pd
from glob import glob
import epiweeks

In [ ]:
pd.set_option('display.max_columns', 150)
pd.set_option('display.max_rows', 500)
pd.options.display.float_format = lambda x: '{:.1f}'.format(x)#.replace(',',';').replace('.',',').replace(';','.')


In [ ]:
from bulletin import default_input, default_output, hoje, ontem, tables
from bulletin.systems import CasosConfirmados
from bulletin.utils.xls_ import fit_cols

In [ ]:
cc = CasosConfirmados()
cc.load(f"cc_{hoje.strftime('%d_%m_%Y')}")
cc.df.set_index('id_notifica',inplace=True)
cc.df.rename(columns={'identificacao':'id'},inplace=True)

In [ ]:
municipios = tables['municipios'].copy()
municipios['regional'] = municipios['rs'].astype(str).str.zfill(2) + 'ª - ' + municipios['regional']
municipios

In [ ]:
municipios.dtypes

In [ ]:
cc.df = cc.df.loc[cc.df['uf_residencia']=='PR']

In [ ]:
def split_date(row):
    row.insert(row.columns.get_loc('data')+1,'ano',row['data'].dt.year)
    row.insert(row.columns.get_loc('data')+2,'mes',row['data'].dt.month)
    row.insert(row.columns.get_loc('data')+3,'se',row['data'].apply(lambda dt: epiweeks.Week.fromdate(dt).week))
    row.insert(row.columns.get_loc('data')+4,'dia',row['data'].dt.day)
    return row

In [ ]:
casos = cc.df.rename(columns={'data_diagnostico':'data','ibge_residencia':'ibge'}).groupby(['ibge','data'])[['id']].count().rename(columns={'id':'casos'})
obitos = cc.df.loc[cc.df['evolucao']==2].rename(columns={'data_cura_obito':'data','ibge_residencia':'ibge'}).groupby(['ibge','data'])[['id']].count().rename(columns={'id':'obitos'})
recuperados = cc.df.loc[cc.df['evolucao']==1].rename(columns={'data_cura_obito':'data','ibge_residencia':'ibge'}).groupby(['ibge','data'])[['id']].count().rename(columns={'id':'recuperados'})
ativos = cc.df.loc[cc.df['evolucao']==3].rename(columns={'data_diagnostico':'data','ibge_residencia':'ibge'}).groupby(['ibge','data'])[['id']].count().rename(columns={'id':'ativos'})

In [ ]:
cora = casos.join([obitos,recuperados,ativos]).fillna(0).astype(int)
cora.reset_index(inplace=True)

In [ ]:
cora = split_date(cora)
cora = cora.join(tables['municipios'].set_index('ibge'), on='ibge')

In [ ]:
fator_incidencia = 10**5

In [ ]:
co = cora.copy() ## casos_obitos
co = co.groupby(['uf','macro','rs','ibge','populacao','ano'])[['casos','obitos']].sum().unstack(-1).fillna(0).sort_index(level=1,axis=0,key=macro_sort)
co.columns = co.columns.set_names('medidas',level=0)

co = co.reorder_levels(order=['ano', 'medidas'], axis=1).sort_index(axis=1)

co_rs = co.reset_index('populacao').groupby(level=[0,1,2]).sum()
co_rs['ibge'] = 'Total'
co_rs = co_rs.set_index(['ibge','populacao'],append=True)

co_macro = co.reset_index('populacao').groupby(level=[0,1]).sum()
co_macro[['rs','ibge']] = 'Total'
co_macro = co_macro.set_index(['rs','ibge','populacao'],append=True)

co_pr = co.reset_index('populacao').groupby(level=[0]).sum()
co_pr[['macro','rs','ibge']] = 'Total'
co_pr = co_pr.set_index(['macro','rs','ibge','populacao'],append=True)

co = pd.concat([co,co_rs,co_macro,co_pr]).sort_index(axis=0,level=1,key=macro_sort)

coefs = (co / co.reset_index('populacao')[['populacao']].values * fator_incidencia)
coefs = coefs.rename({'casos':'coef. incidência','obitos':'coef. mortalidade'},axis=1,level=1)

tx_letalidade = co.loc[:,pd.IndexSlice[:,'obitos']] / co.loc[:,pd.IndexSlice[:,'casos']].values
tx_letalidade = tx_letalidade.rename({'obitos':'tx. letalidade'},axis=1,level=1)

In [ ]:
co_all = co.join(coefs).join(tx_letalidade).sort_index(axis=1).reindex(['casos', 'coef. incidência', 'obitos', 'coef. mortalidade','tx. letalidade'], level='medidas', axis=1)
co_all.sort_index(level=1,axis=0,key=macro_sort)

In [ ]:
writer = pd.ExcelWriter(join(default_output,'medidas_aaa.xlsx'),engine='xlsxwriter')

In [ ]:
macro = co_all.loc[pd.IndexSlice['PR',:,'Total',:],:].droplevel(axis=0,level=['ibge','rs']).drop('Total',axis=0,level=1)
pr = co_all.loc[pd.IndexSlice['PR','Total',:,:],:].droplevel(axis=0,level=['ibge','rs']).rename(index={regional.index.get_level_values('uf')[0]:'Total PR','Total':''})
table = pd.concat([macro,pr])
table.to_excel(writer,'PR')

In [ ]:
for mc in sorted(cora.macro.unique()):
    regional = co_all.loc[pd.IndexSlice[:,mc,:,'Total'],:].droplevel(axis=0,level='ibge').drop('Total',axis=0,level=2)
    regional = regional.rename(index={rs:municipios.set_index('rs').loc[rs,'regional'].iloc[0] for rs in regional.index.get_level_values('rs')})
    
    macro = co_all.loc[pd.IndexSlice[:,regional.index.get_level_values('macro')[0],'Total',:],:].rename(index={regional.index.get_level_values('macro')[0]:'Total Macro','Total':''}).droplevel(axis=0,level='ibge')
    
    pr = co_all.loc[pd.IndexSlice[regional.index.get_level_values('uf')[0],'Total',:,:],:].rename(index={regional.index.get_level_values('uf')[0]:'Total PR','Total':''}).droplevel(axis=0,level='ibge')
    
    table = pd.concat([regional,macro,pr])
    table.to_excel(writer,mc)

In [ ]:
for rs in sorted(cora.rs.unique()):
    mun = co_all.loc[pd.IndexSlice[:,:,rs,:],:].drop('Total',axis=0,level=3)
    mun = mun.rename(index={ibge:municipios.set_index('ibge').loc[ibge,'municipio'] for ibge in mun.index.get_level_values('ibge')})
    mun = mun.rename(index={rs:municipios.set_index('rs').loc[rs,'regional'].iloc[0] for rs in mun.index.get_level_values('rs')})

    regional = co_all.loc[pd.IndexSlice[:,:,rs,'Total'],:].rename(index={rs:'Total Regional','Total':''})
    
    macro = co_all.loc[pd.IndexSlice[:,regional.index.get_level_values('macro')[0],'Total',:],:].rename(index={regional.index.get_level_values('macro')[0]:'Total Macro','Total':''})
    
    pr = co_all.loc[pd.IndexSlice[regional.index.get_level_values('uf')[0],'Total',:,:],:].rename(index={regional.index.get_level_values('uf')[0]:'Total PR','Total':''})
    
    table = pd.concat([mun,regional,macro,pr])
    table.to_excel(writer,str(rs))

In [ ]:
writer.save()
# writer = pd.ExcelWriter(join(default_output,'medidas_macro.xlsx'),engine='xlsxwriter')

# for mc in sorted(cora.macro.unique()):
#     regional = co_all.loc[pd.IndexSlice[:,mc,:,'Total'],:].droplevel(axis=0,level='ibge').drop('Total',axis=0,level=2)
#     regional = regional.rename(index={rs:municipios.set_index('rs').loc[rs,'regional'].iloc[0] for rs in regional.index.get_level_values('rs')})
    
#     macro = co_all.loc[pd.IndexSlice[:,regional.index.get_level_values('macro')[0],'Total',:],:].rename(index={regional.index.get_level_values('macro')[0]:'Total Macro','Total':''}).droplevel(axis=0,level='ibge')
    
#     pr = co_all.loc[pd.IndexSlice[regional.index.get_level_values('uf')[0],'Total',:,:],:].rename(index={regional.index.get_level_values('uf')[0]:'Total PR','Total':''}).droplevel(axis=0,level='ibge')
    
#     table = pd.concat([regional,macro,pr])
#     table.to_excel(writer,str(mc))
    
# writer.save()